In [ ]:
from flask import Flask, request, render_template
import pandas as pd

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Get the uploaded CSV file
        file = request.files['file']
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(file)

        # Drop the columns
        df = df.drop(columns=['start', 'end'])

        # Remove duplicates
        df = df.drop_duplicates()

        # Create boolean to delete non recurring meetings + ones with attendee is 1
        mask = (df['attendees'] == 1) | (df['recurring'] == 'no')

        # drop rows captured by boolean
        df.drop(df[mask].index, inplace=True)

        # Drop the recurring column
        df = df.drop(columns=['recurring'])

        # Create a new  1:1's DataFrame with rows where attendees is equal to 2
        new_df = df[df['attendees'] == 2]

        # Move the new 1:1's dataframe to a new file
        new_df.to_csv('1on1s.csv', index=False, index_label='One on Ones')

        # Drop 1:1's from original file
        mask2 = (df['attendees'] == 2)

        df.drop(df[mask2].index, inplace=True)

        # Sort recurring meetings sheet by cadence
        df = df.sort_values(by='recurrence')

        # Sort 1:1's by cadence
        new_df = new_df.sort_values(by='recurrence')

        # Save 1:1's sheet
        new_df.to_csv('1on1s.csv', index=False)

        # Save recurring meetings sheet
        df.to_csv('recurring_mtgs.csv', index=False)

        return 'Success'
    return render_template('home.html')

if __name__ == '__main__':
    app.run()
